In [69]:
import requests
from bs4 import BeautifulSoup
import time
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [49]:
all_items = []
commitcounter = 0
commits = 0
commitsize = 100
# Loop through all the numbers
#for item_number in range(0,6010):
for item_number in range(0,500):
    url = f"https://collections.frick.org/objects/{item_number}"
    r = requests.get(url)
    if r.status_code == 200:
        #time.sleep(1)
        title = None
        date_created = None
        acs_num = None
        creator_url = None
        creator = None
        creator_nationality = None
        medium = None
        locations = None
        tags = []
        #print(r.text)
        soup = BeautifulSoup(r.text, features="html.parser")
        title_div = soup.find_all("div",{'class':'detailField titleField'})
        if len(title_div) == 1:
            title = title_div[0].text.strip()
        else:
            print("Title not found")
        date_created_div = soup.find("div",{'class':'detailField displayDateField'})
        if date_created_div != None:
            date_created = date_created_div.find('span',{'class':'detailFieldValue'})
            if date_created != None:
                date_created = date_created.text.strip()
        acs_num_div = soup.find_all("div",{'class':'detailField invnoField'})
        if len(acs_num_div) == 1:
            acs_num = acs_num_div[0]
            acs_num = acs_num.find("span",{'class':'detailFieldValue'})
            acs_num = acs_num.text.strip()
        creator_div = soup.find('div',{'class':'detailField peopleField'})
        if creator_div != None:
            creator_a_link = creator_div.find('a')
            creator_url = creator_a_link['href']
            creator_data_elements = creator_div.find_all('span')
            if len(creator_data_elements) == 3:
                creator = creator_data_elements[1].text.strip()
                creator_nationality = creator_data_elements[2].text.strip()
            elif len(creator_data_elements) == 2:
                creator = creator_data_elements[1].text.strip()
            elif len(creator_data_elements) == 1:
                creator = creator_data_elements[0].text.strip()
        medium_div = soup.find('div',{'class':'detailField mediumField'})
        if medium_div != None:
            medium = medium_div.find('span',{'class':'detailFieldValue'})
            if medium != None:
                medium = medium.text.strip()
        locations_div = soup.find('div',{'class':'detailField locationsField'})
        if locations_div != None:
            locations = locations_div.find('span',{'class':'loctionFieldLabel'})
            if locations != None:
                locations = locations.text.strip()
        tags_div = soup.find('div',{'class':'detailField collectionsField'})
        if tags_div != None:
            tag_span_elements = tags_div.find_all('span')
            for span in tag_span_elements:
                tags.append(span.text.strip())

        list_item = {
            'url':url
            , 'title':title
            , 'date_created':date_created
            , 'acs_num':acs_num
            , 'creator_url':creator_url
            , 'creator':creator
            , 'creator_nationality':creator_nationality
            , 'medium':medium
            , 'locations':locations
            , 'tags':tags
        }
        all_items.append(list_item)
        if commitcounter == commitsize:
            commits += 1
            print(f"Commit {commits}")
            with open('frick_data.json','w') as frickfile:
                json.dump(all_items,frickfile,indent=2)
            commitcounter = 0
        else:
            commitcounter += 1
with open('frick_data.json','w') as frickfile:
    json.dump(all_items,frickfile,indent=2)
print("Done")


Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
commit
Title not found
Accession number not found
No creator found.
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name not found
Creator name

In [71]:
with open('frick_data.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
# Pull the year from the accession number
df['acs_year'] = (df['acs_num'].astype(str).str[:4])
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('tags')),
                          columns=mlb.classes_,
                          index=df.index))
print(df.shape)
print(df.columns)



(496, 34)
Index(['url', 'title', 'date_created', 'acs_num', 'creator_url', 'creator',
       'creator_nationality', 'medium', 'locations', 'acs_year',
       '18th and 19th Century Furniture', 'Audio Available', 'Ceramics',
       'Clocks', 'Cocktails with a Curator', 'Enamels', 'Furniture',
       'Gilt Bronzes', 'Highlights', 'Painting', 'Paper',
       'Renaissance Furniture', 'Sculpture', 'Silver', 'Subject: Portraits',
       'Subjects: Animals', 'Subjects: At Home', 'Subjects: Flowers',
       'Subjects: Genre', 'Subjects: Seascapes', 'Textiles', 'Virtual Tour',
       'Vitrual Tour Frick Madison: Room 10, East Wall, Center Panel',
       'Vitrual Tour: Frick Madison, Room 19, East Wall'],
      dtype='object')
